In [2]:
# Install required packages
%pip install gradio==5.37.0 -q
%pip install optuna -q
%pip install plotly -q
%pip install xgboost -q
%pip install lightgbm -q
%pip install scikit-learn -q
%pip install pandas -q
%pip install numpy -q
%pip install scipy -q

print("✅ All packages installed successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.1 MB/s eta 0:00:00
✅ All packages installed successfully!


In [ ]:
# Import all required libraries
import gradio as gr
import pandas as pd
import numpy as np
import optuna
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import asyncio
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple, Callable
from dataclasses import dataclass, asdict
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from scipy import stats
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

print("✅ All imports successful!")


In [ ]:
# ==========================================
# AUTONOMOUS AI SYSTEM FOR FLOOD PREDICTION
# ==========================================

# Configuration and Data Classes
@dataclass
class AgentConfig:
    name: str
    enabled: bool = True
    update_frequency: int = 3600
    performance_threshold: float = 0.85
    max_retries: int = 3
    timeout: int = 300

@dataclass
class ModelPerformance:
    model_name: str
    rmse: float
    r2: float
    mae: float
    timestamp: datetime
    cross_val_score: float
    feature_count: int

@dataclass
class DataQualityReport:
    timestamp: datetime
    missing_values: Dict[str, float]
    data_drift_score: float
    feature_correlations: Dict[str, float]
    outlier_percentage: float
    distribution_changes: Dict[str, float]

class BaseAgent(ABC):
    def __init__(self, config: AgentConfig):
        self.config = config
        self.logger = logging.getLogger(f"Agent.{config.name}")
        self.is_running = False
        self.last_execution = None
        self.performance_history = []

    @abstractmethod
    async def execute(self, context: Dict[str, Any]) -> Dict[str, Any]:
        pass

class DataMonitoringAgent(BaseAgent):
    def __init__(self, config: AgentConfig):
        super().__init__(config)
        self.baseline_stats = None
        self.drift_threshold = 0.3

    async def execute(self, context: Dict[str, Any]) -> Dict[str, Any]:
        try:
            current_data = context.get('current_data')
            if current_data is None:
                return {'status': 'no_data', 'action_required': False}

            quality_report = self._assess_data_quality(current_data)
            drift_detected = self._detect_data_drift(current_data)

            return {
                'status': 'completed',
                'quality_report': asdict(quality_report),
                'drift_detected': drift_detected,
                'action_required': drift_detected or quality_report.missing_values.get('total', 0) > 0.1
            }
        except Exception as e:
            self.logger.error(f"Data monitoring failed: {str(e)}")
            return {'status': 'error', 'error': str(e), 'action_required': True}

    def _assess_data_quality(self, data: pd.DataFrame) -> DataQualityReport:
        missing_values = {col: data[col].isnull().sum() / len(data) for col in data.columns}
        missing_values['total'] = data.isnull().sum().sum() / (len(data) * len(data.columns))

        # Calculate outliers using IQR method
        outlier_count = 0
        for col in data.select_dtypes(include=[np.number]).columns:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = ((data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))).sum()
            outlier_count += outliers

        outlier_percentage = outlier_count / (len(data) * len(data.select_dtypes(include=[np.number]).columns))

        # Feature correlations
        numeric_data = data.select_dtypes(include=[np.number])
        correlations = {}
        if len(numeric_data.columns) > 1:
            corr_matrix = numeric_data.corr()
            correlations = {f"{col1}_{col2}": corr_matrix.loc[col1, col2]
                           for col1 in corr_matrix.columns
                           for col2 in corr_matrix.columns if col1 != col2}

        return DataQualityReport(
            timestamp=datetime.now(),
            missing_values=missing_values,
            data_drift_score=0.0,  # Will be calculated in drift detection
            feature_correlations=correlations,
            outlier_percentage=outlier_percentage,
            distribution_changes={}
        )

    def _detect_data_drift(self, current_data: pd.DataFrame) -> bool:
        if self.baseline_stats is None:
            self.baseline_stats = self._compute_baseline_stats(current_data)
            return False

        drift_detected = False
        for col in current_data.select_dtypes(include=[np.number]).columns:
            if col in self.baseline_stats:
                try:
                    # Kolmogorov-Smirnov test
                    ks_stat, p_value = stats.ks_2samp(
                        self.baseline_stats[col]['sample'],
                        current_data[col].dropna()
                    )
                    if p_value < 0.05:  # Significant drift detected
                        drift_detected = True
                        self.logger.warning(f"Data drift detected in column {col} (p-value: {p_value:.4f})")
                except Exception as e:
                    self.logger.error(f"Error detecting drift for {col}: {str(e)}")

        return drift_detected

    def _compute_baseline_stats(self, data: pd.DataFrame) -> Dict[str, Any]:
        stats_dict = {}
        for col in data.select_dtypes(include=[np.number]).columns:
            stats_dict[col] = {
                'mean': data[col].mean(),
                'std': data[col].std(),
                'min': data[col].min(),
                'max': data[col].max(),
                'sample': data[col].dropna().sample(min(1000, len(data))).values
            }
        return stats_dict

print("✅ Data Monitoring Agent defined")


In [ ]:
# SIMPLE COLAB DEPLOYMENT
# Just run this cell and upload your agentic_ai_system.py file when prompted

import os
from google.colab import files

# Upload the agentic_ai_system.py file
print("📁 Please upload your agentic_ai_system.py file:")
uploaded = files.upload()

# Save the uploaded file
for filename in uploaded.keys():
    with open(filename, 'wb') as f:
        f.write(uploaded[filename])
    print(f"✅ Saved {filename}")

# Now import and run the system
exec(open('agentic_ai_system.py').read())

print("🚀 System ready! Now upload and run app.py or create your own interface.")


In [ ]:
# LAUNCH THE AUTONOMOUS AI SYSTEM
# Run this after uploading your files

# Option 1: Run the full Gradio app
%run app.py

# Option 2: Or create a simple demo here
def create_simple_demo():
    """Create a simple demo of the autonomous system"""

    # Generate sample flood prediction data
    np.random.seed(42)
    n_samples = 1000

    sample_data = pd.DataFrame({
        'MonsoonIntensity': np.random.normal(5, 2, n_samples),
        'TopographyDrainage': np.random.normal(3, 1, n_samples),
        'RiverManagement': np.random.normal(2, 0.5, n_samples),
        'Deforestation': np.random.normal(1, 0.3, n_samples),
        'Urbanization': np.random.normal(2, 0.8, n_samples),
        'ClimateChange': np.random.normal(3, 1, n_samples),
        'DamsQuality': np.random.normal(4, 1.2, n_samples),
        'Siltation': np.random.normal(2, 0.6, n_samples),
        'AgriculturalPractices': np.random.normal(3, 0.9, n_samples),
        'Encroachments': np.random.normal(1, 0.4, n_samples),
        'IneffectiveDisasterPreparedness': np.random.normal(2, 0.7, n_samples),
        'DrainageSystems': np.random.normal(3, 1, n_samples),
        'CoastalVulnerability': np.random.normal(2, 0.8, n_samples),
        'Landslides': np.random.normal(1, 0.5, n_samples),
        'Watersheds': np.random.normal(4, 1.5, n_samples),
        'DeterioratingInfrastructure': np.random.normal(2, 0.6, n_samples),
        'PopulationScore': np.random.normal(3, 1, n_samples),
        'WetlandLoss': np.random.normal(2, 0.7, n_samples),
        'InadequatePlanning': np.random.normal(2, 0.8, n_samples),
        'PoliticalFactors': np.random.normal(1, 0.3, n_samples)
    })

    # Create target variable (FloodProbability)
    sample_data['FloodProbability'] = (
        0.3 * sample_data['MonsoonIntensity'] +
        0.2 * sample_data['TopographyDrainage'] +
        0.1 * sample_data['ClimateChange'] +
        0.1 * sample_data['Deforestation'] +
        0.1 * sample_data['Urbanization'] +
        0.2 * sample_data['IneffectiveDisasterPreparedness'] +
        np.random.normal(0, 0.5, n_samples)
    )

    # Clip to [0, 1] range
    sample_data['FloodProbability'] = np.clip(sample_data['FloodProbability'], 0, 1)

    return sample_data

# Create demo data
demo_data = create_simple_demo()
print(f"✅ Created demo dataset with {len(demo_data)} samples")
print(f"📊 Features: {list(demo_data.columns[:-1])}")
print(f"🎯 Target: FloodProbability (range: {demo_data['FloodProbability'].min():.3f} - {demo_data['FloodProbability'].max():.3f})")

print("\\n🚀 Ready to run autonomous system!")
print("\\n📖 Instructions:")
print("1. Upload agentic_ai_system.py and app.py files")
print("2. Run '%run app.py' to launch the full interface")
print("3. Or manually run the orchestrator with the demo data")
